## Linking in Spark



In [1]:
from splink.spark.jar_location import similarity_jar_location

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import types

conf = SparkConf()
# This parallelism setting is only suitable for a small toy example
conf.set("spark.driver.memory", "12g")
conf.set("spark.default.parallelism", "16")


# Add custom similarity functions, which are bundled with Splink
# documented here: https://github.com/moj-analytical-services/splink_scalaudfs
path = similarity_jar_location()
conf.set("spark.jars", path)

sc = SparkContext.getOrCreate(conf=conf)

spark = SparkSession(sc)
spark.sparkContext.setCheckpointDir("./tmp_checkpoints")

23/06/22 16:32:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
import pandas as pd 
df = spark.read.parquet("../../data/historical_figures_with_errors_50k.parquet", header=True)

In [3]:
import splink.spark.comparison_library as cl
import splink.spark.comparison_template_library as ctl

settings = {
    "link_type": "dedupe_only",
    "comparisons": [
        ctl.name_comparison("first_name", term_frequency_adjustments=True),
        ctl.name_comparison("surname", term_frequency_adjustments=True),
        cl.damerau_levenshtein_at_thresholds("dob", [1, 2]),
        ctl.postcode_comparison("postcode_fake"),
        cl.exact_match("birth_place", term_frequency_adjustments=True),
        cl.exact_match("occupation",  term_frequency_adjustments=True),
    ],
    "blocking_rules_to_generate_predictions": [
        "l.first_name = r.first_name",
        "l.surname = r.surname",
    ],
    "retain_matching_columns": True,
    "retain_intermediate_calculation_columns": True,
    "em_convergence": 0.01
}

In [4]:
from splink.spark.linker import SparkLinker
linker = SparkLinker(df, settings)
deterministic_rules = [
    "l.first_name = r.first_name and l.surname = r.surname and l.dob = r.dob",
    "substr(l.first_name,1,2) = substr(r.first_name,1,2) and l.surname = r.surname and substr(l.postcode_fake,1,2) = substr(r.postcode_fake,1,2)",
    "l.dob = r.dob and l.postcode_fake = r.postcode_fake",
]

linker.estimate_probability_two_random_records_match(deterministic_rules, recall=0.6)


/Users/rosskennedy/Library/r-miniconda/lib/python3.9/site-packages/pyspark/sql/dataframe.py:169: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
/Users/rosskennedy/Library/r-miniconda/lib/python3.9/site-packages/pyspark/sql/pandas/utils.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):
/Users/rosskennedy/Library/r-miniconda/lib/python3.9/site-packages/pyspark/sql/pandas/utils.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):
/Users/rosskennedy/Library/r-miniconda/lib/python3.9/site-packages/pyspark/sql/pandas/utils.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pand

In [5]:
linker.estimate_u_using_random_sampling(max_pairs=5e5)

----- Estimating u probabilities using random sampling -----
23/06/22 16:32:33 WARN DataSource: All paths were ignored:                      
  file:/Users/rosskennedy/splink_demos/examples/spark/tmp_checkpoints/13767a8c-fd44-459d-aa74-bd36ad1c7687/__splink__df_concat_with_tf_8decc3164
/Users/rosskennedy/Library/r-miniconda/lib/python3.9/site-packages/pyspark/sql/pandas/utils.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):
/Users/rosskennedy/Library/r-miniconda/lib/python3.9/site-packages/pyspark/sql/pandas/utils.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):
/Users/rosskennedy/Library/r-miniconda/lib/python3.9/site-packages/pyspark/sql/pandas/utils.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packagin

In [6]:
training_blocking_rule = "l.first_name = r.first_name and l.surname = r.surname"
training_session_fname_sname = linker.estimate_parameters_using_expectation_maximisation(training_blocking_rule)

training_blocking_rule = "l.dob = r.dob"
training_session_dob = linker.estimate_parameters_using_expectation_maximisation(training_blocking_rule)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l.first_name = r.first_name and l.surname = r.surname

Parameter estimates will be made for the following comparison(s):
    - dob
    - postcode_fake
    - birth_place
    - occupation

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - first_name
    - surname
23/06/22 16:32:47 WARN DataSource: All paths were ignored:                      
  file:/Users/rosskennedy/splink_demos/examples/spark/tmp_checkpoints/13767a8c-fd44-459d-aa74-bd36ad1c7687/__splink__df_comparison_vectors_610743d32
/Users/rosskennedy/Library/r-miniconda/lib/python3.9/site-packages/pyspark/sql/pandas/utils.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):
/Users/rosskennedy/Library/r-miniconda/lib/python3.9/site-packages/p

In [7]:
results = linker.predict(threshold_match_probability=0.9)

23/06/22 16:33:33 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [ ]:
results.as_pandas_dataframe(limit=5)

/Users/rosskennedy/Library/r-miniconda/lib/python3.9/site-packages/pyspark/sql/pandas/utils.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):
/Users/rosskennedy/Library/r-miniconda/lib/python3.9/site-packages/pyspark/sql/pandas/utils.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):


,match_weight,match_probability,unique_id_l,unique_id_r,first_name_l,first_name_r,gamma_first_name,bf_first_name,surname_l,surname_r,...,gamma_city,tf_city_l,tf_city_r,bf_city,bf_tf_adj_city,email_l,email_r,gamma_email,bf_email,match_key
0,6.988837,0.992188,190,192,Isla,Isla,4,85.028963,None,Pricie,...,0,0.212792,0.00123,0.462959,1.000000,iprice@carr.com,iprice@carr.com,4,252.590856,0
1,6.417914,0.988440,47,50,Erin,Erin,4,85.028963,Rogers,Roers,...,-1,0.212792,NaN,1.000000,1.000000,e.rogers3@hopkins.org,None,-1,1.000000,0
2,12.767179,0.999857,691,693,Olivia,Olivia,4,85.028963,None,King,...,1,0.022140,0.02214,10.201219,2.490832,olivia.k1@howell.com,olivia.k1@howell.com,4,252.590856,0
3,18.015238,0.999996,819,820,Logan,Logan,4,85.028963,Morgan,Morgan,...,0,0.025830,0.00123,0.462959,1.000000,loganmorgan43@icbride-kmng.com,loganmorgan43@mcbride-king.com,3,212.903339,0
4,11.392229,0.999628,165,166,Elizabeth,Elizabeth,4,85.028963,Wright,Wright,...,-1,NaN,0.01599,1.000000,1.000000,None,elizabeth.w@sullivan-anderson.info,-1,1.000000,0
